In [1]:
import os
import glob
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

#불러오기,결측값제거
df = pd.read_csv('./contest.csv',sep=',')
df = df.dropna()

#시상규모 범주분포화
df['시상규모(만원)'] = df['prize'].apply(lambda x: (x.replace('억', '0000')))          #1억 = 10000만원으로 변경
df["시상규모(만원)"]=df["시상규모(만원)"].str.extract(r'(\d+)')                           #숫자만남기기
df["시상규모(만원)"] = df["시상규모(만원)"].fillna(0).astype(int)                  #NaN(미정)값 0으로 변경                   
df=df.astype({"시상규모(만원)":'int'})
prize_bins=[0,1,251,551,280000000]                        #범주화  0:인원미정/1-560(만원):소/561-1075(만원):중/1076-280000000(만원):대
prize_scale=["시상규모미정","소","중","대"]               
df["시상 규모"]=pd.cut(df["시상규모(만원)"],bins=prize_bins,right=False,labels=prize_scale) #분포범주화



#공모분야 더미변수화
df['field'] = df['field'].replace('기타', '공모분야_기타')                                                 #'기타'열이 중복됨으로 교체 
df = df.join(df["field"].str.split('/', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#기업형태 더미변수화                                         
df['company_type'] = df['company_type'].replace('기타', '기업형태_기타')                                                 #'기타'열이 중복됨으로 교체 
df = df.join(df["company_type"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#참여대상 더미변수화
df = df.join(df["target"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#활동혜택 더미변수화
df = df.join(df["benefit"].str.split(', ', expand=True).stack().str.get_dummies().groupby(level=0).sum())

#열제거
df=df.drop(["contest_name","company_name","prize","period"],axis='columns')

#열위치조정
df = df.reindex(columns=["idx","company_type","공공기관/공기업","기업형태_기타","대기업","비영리단체/협회/재단",
                         "중견기업","중소기업","field","UCC","건설","건축","게임","공모분야_기타","공예",
                         "공학","과학","광고","기획","네이밍","디자인","마케팅","만화","문학","사진","순수미술",
                         "슬로건","시나리오","아이디어","영상","예체능","인테리어","창업","캐릭터","패션","학술",
                         "target","대상 제한 없음","대학생","직장인/일반인","청소년","benefit","기타","상장 수여","실제상용화",
                         "인턴/정규직채용","입사시 가산점","전시기회","해외연수","시상규모(만원)","시상 규모"])


#csv파일저장
df=df.to_csv('contest_process.csv',encoding = 'euc-kr')

In [45]:
df= df.replace(0, np.nan)
df.describe().dropna()
print('\n')
print("시상규모(만원) 기초통계량:\n",df["시상규모(만원)"].describe())



시상규모(만원) 기초통계량:
 count        139.00
mean      202671.15
std      2374824.55
min           15.00
25%          275.00
50%          560.00
75%         1075.00
max     28000000.00
Name: 시상규모(만원), dtype: float64


In [44]:
pd.options.display.float_format = '{:.2f}'.format
df= df.replace(0, np.nan)
df = df.fillna(0)  # NaN 값을 0으로 채우기
print('\n')
print("시상규모(만원) 기초통계량:\n",df["시상규모(만원)"].describe())


TypeError: Cannot setitem on a Categorical with a new category (0), set the categories first